In [1]:
import pandas as pd
from pyexpat import features

data = pd.read_parquet('../cache/encoded_99q_scaled.parquet')
data = data.drop(columns=['Impact_bin', 'AbsChange', 'pct_change_24h', 'pct_change_15min'])
data.head()

,Timestamp,Actor1Country,Actor1GeoCountry,Actor1Type,Actor2Country,Actor2GeoCountry,Actor2Type,ActionCountry,EventType,GoldsteinScale,NumSources,NumArticles,AvgTone,Magnitude,Impact,pct_change_30min
Date,,,,,,,,,,,,,,,,
2019-01-01,-1.530752,9,13,9,13,11,9,11,10,0.605751,0.889228,2.004333,1.164239,0.375252,0.714246,-0.226363
2019-01-01,-1.530752,9,13,9,13,11,9,11,10,0.731167,0.367070,0.597142,0.468568,-0.716820,0.425568,-0.226363
2019-01-01,-1.530752,9,13,9,13,11,9,11,10,0.563945,4.022178,1.300737,1.004049,0.960180,0.848379,-0.226363
2019-01-01,-1.530752,9,13,9,13,11,9,11,10,-0.523000,0.367070,-0.036094,-0.520616,0.334805,-0.355905,-0.226363
2019-01-01,-1.530752,9,13,9,13,11,9,11,10,0.292209,1.933545,-0.247173,0.921836,-0.100779,0.329342,-0.226363


In [2]:
data.dtypes

Timestamp           float64
Actor1Country         int32
Actor1GeoCountry      int32
Actor1Type            int32
Actor2Country         int32
Actor2GeoCountry      int32
Actor2Type            int32
ActionCountry         int32
EventType             int32
GoldsteinScale      float64
NumSources          float64
NumArticles         float64
AvgTone             float64
Magnitude           float64
Impact              float64
pct_change_30min    float64
dtype: object

In [3]:
# Split the data into train and test sets
train_data = data[data.index.year < 2023]
test_data = data[data.index.year == 2023]

print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)

sample = train_data.tail(10000)
sample.head()

Train data shape: (4249654, 16)
Test data shape: (921887, 16)


,Timestamp,Actor1Country,Actor1GeoCountry,Actor1Type,Actor2Country,Actor2GeoCountry,Actor2Type,ActionCountry,EventType,GoldsteinScale,NumSources,NumArticles,AvgTone,Magnitude,Impact,pct_change_30min
Date,,,,,,,,,,,,,,,,
2022-12-27 01:00:00,1.161151,9,13,9,13,11,9,11,10,-0.523000,-0.677247,-0.106454,0.082086,-0.716820,-0.160537,0.000000
2022-12-27 01:00:00,1.161151,9,13,9,13,11,9,11,10,0.731167,-0.155088,-0.387892,-2.016639,1.902909,1.405324,0.000000
2022-12-27 01:00:00,1.161151,9,13,9,13,11,9,11,10,0.480334,-0.677247,-0.669330,-0.678348,0.073454,0.515962,0.000000
2022-12-27 01:15:00,1.161170,9,13,9,13,11,9,11,10,0.104084,-0.677247,-0.106454,0.405894,-1.108846,0.096066,-0.029688
2022-12-27 01:15:00,1.161170,2,4,9,13,3,9,4,10,0.480334,-0.155088,-0.387892,-0.706995,0.325471,0.580113,-0.029688


In [4]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Quantifies multicollinearity
vif_data = pd.DataFrame()
vif_data["Variable"] = sample.columns
vif_data["VIF"] = [variance_inflation_factor(sample.values, i) for i in range(sample.shape[1])]
print(vif_data)

            Variable          VIF
0          Timestamp  2044.879043
1      Actor1Country   193.521757
2   Actor1GeoCountry   165.545638
3         Actor1Type   877.957511
4      Actor2Country   213.618694
5   Actor2GeoCountry   146.489644
6         Actor2Type   775.696064
7      ActionCountry   194.181857
8          EventType    15.299656
9     GoldsteinScale     4.798859
10        NumSources     1.333725
11       NumArticles     1.303256
12           AvgTone     1.862013
13         Magnitude     2.340180
14            Impact     4.974763
15  pct_change_30min     1.005640


### Vector Autoregression (VAR)

In [5]:
import statsmodels.api as sm
from statsmodels.tsa.api import VAR

model = VAR(sample)
results = model.fit(maxlags=15, ic='aic') # Fit with automatic lag order selection based on AIC
lag_order = results.k_ar
predictions = results.forecast(sample.values[-lag_order:], steps=5) # Forecast 5 steps ahead
print(predictions)

C:\Users\leo-mart\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


[[ 1.17029975e+00  8.94934196e+00  1.27236856e+01  8.96695347e+00
   1.30348167e+01  1.09387702e+01  8.97698238e+00  1.07606560e+01
   9.69531217e+00  1.21975588e-02 -2.79608514e-01 -1.51786627e-01
   2.22791905e-01 -3.01620065e-01  2.03298186e-03 -1.82967353e-02]
 [ 1.17030088e+00  8.96032483e+00  1.27255447e+01  8.99521480e+00
   1.29816243e+01  1.09779270e+01  8.97662315e+00  1.07743490e+01
   9.89225659e+00  4.79441676e-02 -2.24129669e-01 -6.87135595e-02
   1.79035586e-01 -1.90289034e-01  6.15890685e-02 -1.98621729e-02]
 [ 1.17030214e+00  8.99202937e+00  1.26400020e+01  8.99073216e+00
   1.30161442e+01  1.09952493e+01  8.98270102e+00  1.07330021e+01
   9.92722907e+00 -2.78012933e-03 -2.50177418e-01 -1.62511501e-01
   2.30497986e-01 -2.70455440e-01 -4.80875984e-03 -1.89104160e-02]
 [ 1.17030333e+00  8.97824926e+00  1.26386877e+01  8.98713445e+00
   1.30196361e+01  1.09865047e+01  8.97757474e+00  1.07181917e+01
   9.87856510e+00 -1.57157516e-02 -2.09153073e-01 -1.47691715e-01
   1.63

### dynamic factor model (DFM)

In [6]:
from statsmodels.tsa.statespace.dynamic_factor import DynamicFactor

# Fit a dynamic factor model
model = DynamicFactor(sample, k_factors=1, factor_order=1)
results = model.fit()
print(results.summary())

C:\Users\leo-mart\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\leo-mart\AppData\Local\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                                                                                                                             Statespace Model Results                                                                                                                                             
Dep. Variable:     ['Timestamp', 'Actor1Country', 'Actor1GeoCountry', 'Actor1Type', 'Actor2Country', 'Actor2GeoCountry', 'Actor2Type', 'ActionCountry', 'EventType', 'GoldsteinScale', 'NumSources', 'NumArticles', 'AvgTone', 'Magnitude', 'Impact', 'pct_change_30min']   No. Observations:                10000
Model:                                                                                                                                                                                                                                  DynamicFactor(factors=1, order=1)   Log Likelihood             -426711.135
Date:                                                                          

In [7]:
forecast_steps = 5
forecast = results.get_forecast(steps=forecast_steps)
# Extract the predicted values from the forecast result
predicted_values = forecast.predicted_mean
# Or for the full prediction including uncertainty (confidence intervals)
prediction_conf_int = forecast.conf_int()

# Display the predicted values and the confidence intervals
print("Predicted Values:")
print(predicted_values)

print("\nConfidence Intervals for Predictions:")
print(prediction_conf_int)

Predicted Values:
          Timestamp  Actor1Country  Actor1GeoCountry    Actor1Type  \
10000  1.905504e-05   1.677288e-03      6.459363e-03  3.584767e-05   
10001  1.621922e-06   1.427670e-04      5.498063e-04  3.051272e-06   
10002  1.380543e-07   1.215200e-05      4.679827e-05  2.597174e-07   
10003  1.175087e-08   1.034351e-06      3.983363e-06  2.210656e-08   
10004  1.000208e-09   8.804161e-08      3.390548e-07  1.881660e-09   

       Actor2Country  Actor2GeoCountry    Actor2Type  ActionCountry  \
10000   1.545807e-03      3.131220e-03  9.752733e-05   5.341236e-03   
10001   1.315756e-04      2.665224e-04  8.301305e-06   4.546339e-04   
10002   1.119942e-05      2.268578e-05  7.065883e-07   3.869741e-05   
10003   9.532691e-07      1.930962e-06  6.014320e-08   3.293836e-06   
10004   8.114011e-08      1.643591e-07  5.119253e-09   2.803639e-07   

          EventType  GoldsteinScale    NumSources   NumArticles       AvgTone  \
10000  1.500901e-04    8.621478e-03 -2.863274e-03 -3.

C:\Users\leo-mart\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\leo-mart\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


### Vector Autoregression Moving-Average (VARMA)

In [8]:
import statsmodels.api as sm
from statsmodels.tsa.statespace.varmax import VARMAX

# Assuming 'data' is a pandas DataFrame with multiple time series columns
model = VARMAX(sample.drop(columns=['Actor1Country', 'Actor2Country', 'NumSources', 'NumArticles', 'AvgTone', 'GoldsteinScale']), order=(1, 1))  # (p, q) - autoregressive and moving average orders
results = model.fit(disp=False)
predictions = results.forecast(steps=5)
print(predictions)

C:\Users\leo-mart\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\statespace\varmax.py:160: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
C:\Users\leo-mart\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\leo-mart\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\statespace\varmax.py:373: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\leo-mart\AppData\Local\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_r

       Timestamp  Actor1GeoCountry  Actor1Type  Actor2GeoCountry  Actor2Type  \
10000   1.165967         19.814492    8.856589         13.165950   10.874292   
10001   1.165077         11.974672    9.051145         10.703791    8.888771   
10002   1.165227         11.974970    9.051628         10.701237    8.888559   
10003   1.165232         11.974891    9.051635         10.701216    8.888546   
10004   1.165233         11.974888    9.051636         10.701215    8.888545   

       ActionCountry  EventType  Magnitude    Impact  pct_change_30min  
10000      23.460156  -6.521625 -14.624744  1.224671          0.137775  
10001       9.842941  10.553419   0.579796  0.056872          0.157031  
10002       9.841836  10.549444   0.580371  0.056227          0.151269  
10003       9.841735  10.549539   0.580459  0.056230          0.151202  
10004       9.841731  10.549543   0.580462  0.056230          0.151202  


C:\Users\leo-mart\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
C:\Users\leo-mart\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
